In [1]:
#!pip install xgboost

In [2]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.2.2

In [ ]:
import numpy as np
import pandas as pd
import datetime
import time
from sklearn.model_selection import train_test_split


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import tree

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Metrics to use other than the accuracy (Success rate)
# https://scikit-learn.org/stable/modules/classes.html?highlight=metric#module-sklearn.metrics

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from itertools import cycle
import matplotlib.pyplot as plt 

In [ ]:
X_tr = pd.read_csv('data/CombinedBestFeatures.csv')
X_test = pd.read_csv('data/X_VALIDATION_SET_ENCODED.csv')
Y_tr = pd.read_csv('data/Y_TRAINING_SET_EDIT.csv')
y_test = pd.read_csv('data/Y_VALIDATION_SET_EDIT.csv')

In [ ]:
X_tr = X_tr.drop(columns=['Unnamed: 0'])
X_test = X_test.drop(columns=['Unnamed: 0'])
Y_tr = Y_tr.drop(columns=['Unnamed: 0'])
y_test = y_test.drop(columns=['Unnamed: 0'])

In [ ]:
new_column_names = [col.replace('[', '_').replace(']', '_').replace('<', 'less_than') for col in X_tr.columns]
column_mapping = dict(zip(X_tr.columns, new_column_names))
X_tr = X_tr.rename(columns=column_mapping)
new_column_names = [col.replace('[', '_').replace(']', '_').replace('<', 'less_than') for col in X_test.columns]
column_mapping = dict(zip(X_test.columns, new_column_names))
X_test = X_test.rename(columns=column_mapping)


In [ ]:
X_test

In [ ]:
common_columns = X_tr.columns
X_test = X_test[common_columns]
X_test.to_csv("data/Xtest_common_featureSelection.csv")

In [ ]:
X_test


In [ ]:
def results(model, X_train, y_train, X_test, y_test, r1,r2,r3):
    r1[type(model).__name__] = []
    r2[type(model).__name__] = []
    r3[type(model).__name__] = []
    model.fit(X_train, y_train)

    print(type(model).__name__)
   
    print("Acc:  ", model.score(X_test, y_test))

    y_pred = model.predict(X_test)

    y_test_bin = label_binarize(y_test, classes=np.unique(y_test))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
        
    #class0
    fpr[0], tpr[0], _ = roc_curve(y_test_bin[:, 0], model.predict_proba(X_test)[:, 0])
    roc_auc[0] = auc(fpr[0], tpr[0])
    r1[type(model).__name__] = [fpr[0], tpr[0], roc_auc[0]]
    #class1
    fpr[1], tpr[1], _ = roc_curve(y_test_bin[:, 1], model.predict_proba(X_test)[:, 1])
    roc_auc[1] = auc(fpr[1], tpr[1])
    r2[type(model).__name__] = [fpr[1], tpr[1], roc_auc[1]]
    #class2
    fpr[2], tpr[2], _ = roc_curve(y_test_bin[:, 2], model.predict_proba(X_test)[:, 2])
    roc_auc[2] = auc(fpr[2], tpr[2])
    r3[type(model).__name__] = [fpr[2], tpr[2], roc_auc[2]]      

    prec = precision_score(y_test, y_pred, average=None)
    
    print("Prec: ", prec)

    sens = recall_score(y_test, y_pred, average=None)
    print("Sens: ", sens)

    f1 = f1_score(y_test, y_pred, average=None)
    print("F1: ", f1)

    print(classification_report(y_test, y_pred))


    res = {"Model_Name": type(model).__name__, "Accuracy": model.score(X_test, y_test),
           "Precision": prec, "Sensitivity": sens, "F1": f1, "ROC_AUC": roc_auc}



    return res

In [ ]:
model = SVC(probability=True)
model.fit(X_tr, Y_tr)

In [ ]:
acc = model.score(X_test, y_test)
print("Acc:  ",acc)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'C':[0.1,1,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6],'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
grid = GridSearchCV(SVC(),param_grid)
grid.fit(X_tr,Y_tr)

C:\Users\diyac\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diyac\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diyac\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\diyac\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [ ]:
print(grid.best_params_)
print(grid.score(X_test,y_test))

# With the original data

In [70]:

roc_1 = {} 
roc_2 = {}
roc_3 = {}


dict_models = {}
dict_models['SVC'] = results(model, X_tr, Y_tr, X_test, y_test, roc_1, roc_2, roc_3)

df = pd.DataFrame.from_dict(dict_models, orient='index')
df




C:\Users\diyac\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:
def disp_roc(roc_1,roc_2,roc_3):
    # Plot ROC curves for each class separately
    plt.figure(figsize=(15, 5))

    # Class 0 ROC curves
    plt.subplot(1, 3, 1)
    for model_name, values in roc_1.items():
        fpr, tpr, roc_auc = values
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Class 0')
    plt.legend()

    # Class 1 ROC curves
    plt.subplot(1, 3, 2)
    for model_name, values in roc_2.items():
        fpr, tpr, roc_auc = values
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Class 1')
    plt.legend()

    # Class 2 ROC curves
    plt.subplot(1, 3, 3)
    for model_name, values in roc_3.items():
        fpr, tpr, roc_auc = values
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Class 2')
    plt.legend()

    plt.tight_layout()
    plt.show()
